In [16]:
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv("dataset/borvo_HW_map.csv")

# Konversi ke grid dictionary
grid = {}
for _, row in df.iterrows():
    grid[(int(row["i"]), int(row["j"]))] = {"H": row["H"], "W": row["W"]}

In [17]:
# Fungsi ambil tetangga 8 arah (termasuk diagonal)
def get_all_neighbors(pos):
    i, j = pos
    directions = [
        (-1, -1), (-1, 0), (-1, 1),
        (0, -1),           (0, 1),
        (1, -1), (1, 0), (1, 1)
    ]
    neighbors = [(i + di, j + dj) for di, dj in directions]
    return [n for n in neighbors if n in grid]

# Fungsi hitung skor W/E
def get_score(current_pos, neighbor_pos):
    H1 = grid[current_pos]["H"]
    H2 = grid[neighbor_pos]["H"]
    S = H1 - H2

    if S > 0.5:
        return -np.inf  # Tidak aman → skip

    E = 1 + 2 * abs(S)
    W = grid[neighbor_pos]["W"]
    return W / E

In [18]:
# Fungsi Hill Climbing dengan Horizon (Lookahead Depth)
def hill_climbing_with_horizon(start, depth=3):
    if start not in grid:
        print(f"Titik {start} tidak ditemukan dalam dataset.")
        return []

    current = start
    path = [current]
    visited = set([start])

    def dfs(current_path, total_score, remaining_depth):
        current_node = current_path[-1]
        if remaining_depth == 0:
            return (total_score, current_path)

        best = (total_score, current_path)
        for neighbor in get_all_neighbors(current_node):
            if neighbor in current_path:
                continue
            score = get_score(current_node, neighbor)
            if score == -np.inf:
                continue
            result = dfs(current_path + [neighbor], total_score + score, remaining_depth - 1)
            if result[0] > best[0]:
                best = result
        return best

    while True:
        # Temukan langkah pertama dari jalur terbaik hingga kedalaman tertentu
        best_score = -np.inf
        best_next = None
        neighbors = get_all_neighbors(current)

        for neighbor in neighbors:
            if neighbor in visited:
                continue
            score = get_score(current, neighbor)
            if score == -np.inf:
                continue
            result_score, result_path = dfs([current, neighbor], score, depth - 1)
            if result_score > best_score:
                best_score = result_score
                best_next = result_path[1]  # langkah pertama

        if best_next is None:
            break

        current = best_next
        path.append(current)
        visited.add(current)

    return path


In [20]:
# Titik uji coba
test_points = [
    (540, 648),
    (20, 502),
    (588, 1000)
]

# Jalankan algoritma untuk semua titik uji
for idx, point in enumerate(test_points, 1):
    print(f"\n=== Titik {idx} - Start di {point} ===")
    path = hill_climbing_with_horizon(point)
    print(f"Jumlah langkah: {len(path)}")
    print("Jalur:")
    for p in path:
        print(p)


=== Titik 1 - Start di (540, 648) ===
Jumlah langkah: 136
Jalur:
(540, 648)
(539, 648)
(540, 649)
(540, 650)
(539, 649)
(538, 648)
(537, 649)
(537, 648)
(538, 647)
(539, 646)
(539, 645)
(539, 644)
(539, 643)
(538, 642)
(537, 642)
(538, 641)
(537, 640)
(538, 640)
(538, 639)
(539, 638)
(540, 637)
(541, 636)
(542, 637)
(542, 636)
(543, 637)
(542, 638)
(541, 639)
(540, 639)
(540, 638)
(539, 639)
(539, 640)
(540, 641)
(541, 640)
(542, 639)
(541, 638)
(541, 637)
(540, 636)
(540, 635)
(539, 634)
(538, 634)
(537, 635)
(536, 636)
(535, 637)
(536, 638)
(535, 639)
(534, 638)
(534, 637)
(533, 636)
(532, 635)
(531, 634)
(530, 633)
(530, 632)
(530, 631)
(531, 630)
(530, 629)
(530, 630)
(531, 631)
(531, 632)
(532, 632)
(532, 631)
(532, 630)
(533, 630)
(534, 629)
(533, 628)
(534, 627)
(533, 626)
(532, 626)
(533, 627)
(532, 627)
(532, 628)
(532, 629)
(533, 629)
(534, 630)
(534, 631)
(534, 632)
(533, 633)
(532, 633)
(531, 633)
(530, 634)
(529, 634)
(528, 635)
(527, 634)
(527, 635)
(526, 634)
(525, 634)